In [3]:
import pymongo
import pandas as pd
import praw
import re
import time
import datetime
from datetime import datetime, timedelta

In [33]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db
companylist = db.companylist
secret = db.secret.find_one()['Secret']

In [5]:
df = pd.DataFrame.from_records(companylist.find())

In [6]:
reddit = praw.Reddit(client_id = '-wpcPIbA7bhlpw', client_secret = secret, user_agent = 'sentiment')

In [7]:
wantedTags = ['DD', 'Technicals', 'Fundamentals', 'Discussion', 'YOLO', 'Stocks']
subreddit = reddit.subreddit('wallstreetbets')
postlist = []

In [17]:
for post in subreddit.hot(limit = 1000):
    if datetime.today().weekday() == 0:
        if len(postlist) < 100:
            postTime = post.created
            postDate = datetime.fromtimestamp(postTime)
            
            currentTime = datetime.utcnow()
            
            postDelta = currentTime - postDate

            if postDelta.days < 3 and post.link_flair_text in wantedTags:
                postlist.append(post)
        else:
            break
    
    else:
        if len(postlist) < 100:
            postTime = post.created
            postDate = datetime.utcfromtimestamp(postTime)

            currentTime = datetime.utcnow()

            postDelta = currentTime - postDate
            postDelta = str(postDelta)

            if 'day' not in postDelta and post.link_flair_text in wantedTags:
                postlist.append(post)
        else:
            break

In [18]:
postDf = pd.DataFrame(columns = ['Inserted Date','Title','Body'])

insertedDate = currentTime
titles = [post.title for post in postlist]
bodies = [post.selftext for post in postlist]
tags = [post.link_flair_text for post in postlist]

postDf['Title'] = titles
postDf['Body'] = bodies
postDf['Inserted Date'] = insertedDate
postDf['Tag'] = tags

In [19]:
def cleanStrings(string):
    return re.sub('[^a-zA-Z0-9./$]+', ' ',string)

postDf['Title'] = postDf['Title'].apply(cleanStrings)
postDf['Body'] = postDf['Body'].apply(cleanStrings)

In [28]:
start = db['Inserted Date'][0]

In [34]:
db = pd.DataFrame.from_records(db.threads.find({'Inserted Date': {'$gt': start}} ))